# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 10 2022 12:26PM,239336,10,MET211133,"Methapharm, Inc.",Released
1,Jun 10 2022 12:18PM,239333,10,0085754838,ISDIN Corporation,Released
2,Jun 10 2022 12:18PM,239333,10,0085754837,ISDIN Corporation,Released
3,Jun 10 2022 12:18PM,239333,10,0085754840,ISDIN Corporation,Released
4,Jun 10 2022 12:18PM,239333,10,0085754847,ISDIN Corporation,Released
5,Jun 10 2022 12:18PM,239333,10,0085754844,ISDIN Corporation,Released
6,Jun 10 2022 12:18PM,239333,10,0085754848,ISDIN Corporation,Released
7,Jun 10 2022 12:18PM,239333,10,0085754871,ISDIN Corporation,Released
8,Jun 10 2022 12:18PM,239333,10,0085754870,ISDIN Corporation,Released
9,Jun 10 2022 12:18PM,239333,10,0085754887,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,239331,Released,7
29,239332,Released,4
30,239333,Released,30
31,239334,Released,1
32,239336,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239331,NaN,NaN,7.0
239332,NaN,NaN,4.0
239333,NaN,NaN,30.0
239334,NaN,NaN,1.0
239336,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239258,0.0,0.0,1.0
239261,0.0,1.0,1.0
239286,0.0,0.0,1.0
239291,0.0,0.0,1.0
239292,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239258,0,0,1
239261,0,1,1
239286,0,0,1
239291,0,0,1
239292,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239258,0,0,1
1,239261,0,1,1
2,239286,0,0,1
3,239291,0,0,1
4,239292,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239258,,,1
1,239261,,1,1
2,239286,,,1
3,239291,,,1
4,239292,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 10 2022 12:26PM,239336,10,"Methapharm, Inc."
1,Jun 10 2022 12:18PM,239333,10,ISDIN Corporation
31,Jun 10 2022 12:18PM,239334,10,"Nextsource Biotechnology, LLC"
32,Jun 10 2022 12:12PM,239332,10,Emerginnova
36,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation
43,Jun 10 2022 11:57AM,239328,12,Hush Hush
45,Jun 10 2022 11:56AM,239329,21,"NBTY Global, Inc."
46,Jun 10 2022 11:45AM,239327,21,"NBTY Global, Inc."
47,Jun 10 2022 11:23AM,239326,21,"NBTY Global, Inc."
48,Jun 10 2022 11:17AM,239324,19,"Granules Pharmaceuticals, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 10 2022 12:26PM,239336,10,"Methapharm, Inc.",,,1
1,Jun 10 2022 12:18PM,239333,10,ISDIN Corporation,,,30
2,Jun 10 2022 12:18PM,239334,10,"Nextsource Biotechnology, LLC",,,1
3,Jun 10 2022 12:12PM,239332,10,Emerginnova,,,4
4,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation,,,7
5,Jun 10 2022 11:57AM,239328,12,Hush Hush,,,2
6,Jun 10 2022 11:56AM,239329,21,"NBTY Global, Inc.",,,1
7,Jun 10 2022 11:45AM,239327,21,"NBTY Global, Inc.",,,1
8,Jun 10 2022 11:23AM,239326,21,"NBTY Global, Inc.",,,1
9,Jun 10 2022 11:17AM,239324,19,"Granules Pharmaceuticals, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 12:26PM,239336,10,"Methapharm, Inc.",1,,
1,Jun 10 2022 12:18PM,239333,10,ISDIN Corporation,30,,
2,Jun 10 2022 12:18PM,239334,10,"Nextsource Biotechnology, LLC",1,,
3,Jun 10 2022 12:12PM,239332,10,Emerginnova,4,,
4,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation,7,,
5,Jun 10 2022 11:57AM,239328,12,Hush Hush,2,,
6,Jun 10 2022 11:56AM,239329,21,"NBTY Global, Inc.",1,,
7,Jun 10 2022 11:45AM,239327,21,"NBTY Global, Inc.",1,,
8,Jun 10 2022 11:23AM,239326,21,"NBTY Global, Inc.",1,,
9,Jun 10 2022 11:17AM,239324,19,"Granules Pharmaceuticals, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 12:26PM,239336,10,"Methapharm, Inc.",1,,
1,Jun 10 2022 12:18PM,239333,10,ISDIN Corporation,30,,
2,Jun 10 2022 12:18PM,239334,10,"Nextsource Biotechnology, LLC",1,,
3,Jun 10 2022 12:12PM,239332,10,Emerginnova,4,,
4,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation,7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 12:26PM,239336,10,"Methapharm, Inc.",1.0,NaN,NaN
1,Jun 10 2022 12:18PM,239333,10,ISDIN Corporation,30.0,NaN,NaN
2,Jun 10 2022 12:18PM,239334,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Jun 10 2022 12:12PM,239332,10,Emerginnova,4.0,NaN,NaN
4,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation,7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 12:26PM,239336,10,"Methapharm, Inc.",1.0,0.0,0.0
1,Jun 10 2022 12:18PM,239333,10,ISDIN Corporation,30.0,0.0,0.0
2,Jun 10 2022 12:18PM,239334,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Jun 10 2022 12:12PM,239332,10,Emerginnova,4.0,0.0,0.0
4,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation,7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1435981,52.0,16.0,4.0
12,717895,3.0,12.0,0.0
15,717912,12.0,20.0,6.0
16,717895,21.0,0.0,0.0
19,239324,1.0,0.0,0.0
20,478636,24.0,5.0,0.0
21,2153787,9.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1435981,52.0,16.0,4.0
1,12,717895,3.0,12.0,0.0
2,15,717912,12.0,20.0,6.0
3,16,717895,21.0,0.0,0.0
4,19,239324,1.0,0.0,0.0
5,20,478636,24.0,5.0,0.0
6,21,2153787,9.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,52.0,16.0,4.0
1,12,3.0,12.0,0.0
2,15,12.0,20.0,6.0
3,16,21.0,0.0,0.0
4,19,1.0,0.0,0.0
5,20,24.0,5.0,0.0
6,21,9.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,12,Released,3.0
2,15,Released,12.0
3,16,Released,21.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,4.0,0.0,6.0,0.0,0.0,0.0,0.0
Executing,16.0,12.0,20.0,0.0,0.0,5.0,0.0
Released,52.0,3.0,12.0,21.0,1.0,24.0,9.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,4.0,0.0,6.0,0.0,0.0,0.0,0.0
1,Executing,16.0,12.0,20.0,0.0,0.0,5.0,0.0
2,Released,52.0,3.0,12.0,21.0,1.0,24.0,9.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,4.0,0.0,6.0,0.0,0.0,0.0,0.0
1,Executing,16.0,12.0,20.0,0.0,0.0,5.0,0.0
2,Released,52.0,3.0,12.0,21.0,1.0,24.0,9.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()